In [49]:
import pandas as pd
import numpy as np
from sklearn import metrics, model_selection, svm, inspection
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go

In [50]:
#Gets CSV file and skips metadata
StartData1 = pd.read_csv("data/StartDataCoor11.csv", skiprows = 11)

StartData2 = pd.read_csv("data/StartDataCoor12.csv", skiprows = 11)
StartData2.head()

,PacketCounter,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status
0,1,583947098,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2,583963765,1.0,-0.000076,0.000675,0.000219,-0.150605,0.035473,0.021612,0
2,3,583980432,1.0,-0.000007,0.000628,0.000250,-0.150670,0.035739,0.020749,0
3,4,583997099,1.0,-0.000013,0.000604,0.000275,-0.151207,0.036407,0.020591,0
4,5,584013766,1.0,-0.000053,0.000584,0.000288,-0.151701,0.035671,0.021034,0


In [51]:
#Edit and save data to variable
StartData1Edit = StartData1.drop(columns = ["PacketCounter"])

StartData2Edit = StartData2.drop(columns = ["PacketCounter"])
StartData2Edit.head()

,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status
0,583947098,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,583963765,1.0,-0.000076,0.000675,0.000219,-0.150605,0.035473,0.021612,0
2,583980432,1.0,-0.000007,0.000628,0.000250,-0.150670,0.035739,0.020749,0
3,583997099,1.0,-0.000013,0.000604,0.000275,-0.151207,0.036407,0.020591,0
4,584013766,1.0,-0.000053,0.000584,0.000288,-0.151701,0.035671,0.021034,0


In [52]:
#This is where coordinates can be inputed for each IMU
xO = float(input("What is the x-offset for IMU 2: "))
yO = float(input("What is the y-offset for IMU 2: "))
zO = float(input("What is the z-offset for IMU 2: "))


In [53]:
#This is to change to cartesean coordinates

# Extract quaternion components from the DataFrame
dq_W = StartData1['dq_W'].values
dq_X = StartData1['dq_X'].values
dq_Y = StartData1['dq_Y'].values
dq_Z = StartData1['dq_Z'].values

# Function to compute Cartesian coordinates from quaternions
def quaternion_to_cartesian(w, x, y, z):
    # Convert the quaternion into rotation matrix (example for identity vector rotation)
    x_cart = 2 * (x * z + w * y)
    y_cart = 2 * (y * z - w * x)
    z_cart = 1 - 2 * (x**2 + y**2)
    
    return x_cart, y_cart, z_cart

# Apply the function to each row of the DataFrame
cartesian_coordinates = np.array([quaternion_to_cartesian(w, x, y, z) for w, x, y, z in zip(dq_W, dq_X, dq_Y, dq_Z)])

# Extract x, y, z coordinates
x1 = cartesian_coordinates[:, 0]
y1 = cartesian_coordinates[:, 1]
z1 = cartesian_coordinates[:, 2]

# Now x_coords, y_coords, z_coords contain the computed Cartesian coordinates

# Extract quaternion components from the DataFrame
dq_W = StartData2['dq_W'].values
dq_X = StartData2['dq_X'].values
dq_Y = StartData2['dq_Y'].values
dq_Z = StartData2['dq_Z'].values

# Function to compute Cartesian coordinates from quaternions
def quaternion_to_cartesian(w, x, y, z):
    # Convert the quaternion into rotation matrix (example for identity vector rotation)
    x_cart = 2 * (x * z + w * y)
    y_cart = 2 * (y * z - w * x)
    z_cart = 1 - 2 * (x**2 + y**2)
    
    return x_cart, y_cart, z_cart

# Apply the function to each row of the DataFrame
cartesian_coordinates = np.array([quaternion_to_cartesian(w, x, y, z) for w, x, y, z in zip(dq_W, dq_X, dq_Y, dq_Z)])

# Extract x, y, z coordinates
x2 = cartesian_coordinates[:, 0]
y2 = cartesian_coordinates[:, 1]
z2 = cartesian_coordinates[:, 2]

# Now x_coords, y_coords, z_coords contain the computed Cartesian coordinates


In [55]:
#3D plot of the Euler coordinates

# Create a 3D scatter plot
trace1 = go.Scatter3d(x=x1, y=y1, z=z1, mode='markers', marker=dict(size=5, color='red'), name='IMU 1')
trace2 = go.Scatter3d(x=x2+xO, y=y2+yO, z=z2+zO, mode='markers', marker=dict(size=5, color='blue'), name='IMU 2')

# Set layout for the plot
layout = go.Layout(scene=dict(
    xaxis_title='Euler_X',
    yaxis_title='Euler_Y',
    zaxis_title='Euler_Z'),
    title="Interactive 3D Plot")

# Create the figure
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Show the interactive plot
fig.show()